# SQL Sports Leagues Lab

We are going to build a SQL database to help sports leagues keep track of of their teams and players.  We also will be able to log games and keep track of scores.  In this lab we will utilize all of the tools we learned in the previous lessons and labs.  We will make tables for `leagues`, `teams`, `players`, and `games`, populate these tables with data and make the proper associations, and query from these tables to select interesting information.

## Objectives

1.  Review creating tables, altering tables, inserting data, and querying from tables
2.  Become comfortable working with common data relationships "has many"/"belongs to" and "many-to-many"
3.  Write advanced `SELECT` queries using `JOIN` statements and join tables

### Part 1: `create.sql`

Build the tables that will make up our Sports Leagues database.  Make sure the tables adhere to the following requirements:

1.  `leagues` have a name and "have many" teams
2.  `teams` have a name and "have many" players
3.  `players` have a name and "belong to" a team
4.  `games` have a date of the *date* datatype and a location.  Each date is recorded in a string in the following format: "YYYY-MM-DD"
5.  `teams` have many games and `games` are played by many teams.  Since this relationship is "many-to-many", we will need a join table.  Let's call this join table `team_games`, and it will have foreign keys to track team and game ids.  These foreign keys are responsible for establishing the many-to-many relationship.  `team_games` will also have a column called score that keeps track of that team's score for that particular game.  The important point to remember here is that join tables can have columns that are not foreign keys.
6.  All tables will have an auto-incrementing `PRIMARY KEY` set to the integer data type


### Part 2: `insert.sql`

Once the tables are created, populate the database with data.  Feel free to create fictional leagues, teams, and games.  However, your tables should meet the following requirements.  There should be:

1.  2 leagues
2.  4 teams total, with 2 teams in each league
3.  At least 1 player per team
4.  3 total games, and teams are allowed to play teams from other leagues
5.  Lastly, we will need to log scores and the relevant team and game ids into the `team_games` join table to build out the "many-to-many" relationship

### Part 3: `update.sql`

Two players have decided to change their names.  Fix the `players` table so that the first player's name (id = 1) is switched to "Metta World Peace" and the fourth player's name (id = 4) is changed to "Chad OchoCinco".

### Part 4: `queries.py`

Write queries in the `queries.py` file to get the tests to pass.  We have seeded the database with different data, so don't expect the data your input in the above files to be returned in your queries.  Be prepared to use different sorting and grouping methods and aggregate functions.  Refer to the `seed.sql` file to understand the data populating the database.  Notice we have added a column called `info` to the `games` table.

> **Note:** `select_name_of_player_with_shortest_name`

> This query requires you to select the name of the player with the the shortest name.  Check out the [LENGTH function](https://www.w3schools.com/sql/func_mysql_length.asp).  Is there a sorting method we can use to sort the values returned by the LENGTH function?

## Summary

In this lab we started out by creating tables. We made our tables so that they would have many-to-many, has-many, and belongs-to relationships, enabling us to query information related to records accross tables. We then practiced seeding our database with information on teams, players, and games. Next, we wrote SQL statements that updated records in our database with new names. Finally, we wrote more complex SQL statements that queried and returned information from our database.

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('test.db')
cur = conn.cursor()
def execute_sql(command):
    return cur.execute(command).fetchall()
def create_df(select):
    df = pd.DataFrame(select)
    df.columns = [i[0] for i in cur.description]
    return df

In [83]:
def select_name_of_player_with_shortest_name():
    return """SELECT name FROM players
                WHERE LENGTH(name) = (SELECT MIN(LENGTH(name)) FROM players);
                """

def select_all_new_york_players_names():
    return """SELECT players.name
                FROM players
                JOIN teams ON players.team_id = teams.id
                WHERE teams.name LIKE "%new york%";
                """

def select_team_name_and_total_goals_scored_for_new_york_rangers():
    return """SELECT teams.name, SUM(team_games.score)
                FROM teams JOIN team_games
                ON teams.id = team_games.team_id
                GROUP BY teams.name
                HAVING teams.name = "New York Rangers";
                """

def select_all_games_date_and_info_teams_name_and_score_for_teams_in_nhl():
    return """SELECT games.date, games.info, teams.name, team_games.score
                FROM games JOIN team_games
                ON games.id = team_games.game_id
                JOIN teams ON teams.id = team_games.team_id
                WHERE teams.league_id = (SELECT id FROM leagues WHERE name ="NHL")
                ;
                """

def select_date_info_and_total_points_for_highest_scoring_nba_game():
    return """SELECT games.date, games.info, MAX(team_games.score) 
                FROM games JOIN team_games 
                ON games.id = team_games.game_id
                WHERE team_games.team_id = 
                (SELECT teams.id 
                FROM teams JOIN leagues 
                ON teams.league_id = leagues.id 
                WHERE leagues.name = "NBA");
                """


In [84]:
execute_sql(select_name_of_player_with_shortest_name())

[('Pavel Bure',)]

In [85]:
execute_sql(select_all_new_york_players_names())

[('Metta World Peace',),
 ('Jeff Beukeboom',),
 ('Mark Messier',),
 ('Chad OchoCinco',),
 ('Alexei Kovalev',),
 ('Mike Richter',),
 ('Patrick Ewing',),
 ('Charles Oakley',),
 ('John Starks',),
 ('Anthony Mason',),
 ('Derek Harper',)]

In [86]:
execute_sql(select_team_name_and_total_goals_scored_for_new_york_rangers())

[('New York Rangers', 21)]

In [87]:
execute_sql(select_all_games_date_and_info_teams_name_and_score_for_teams_in_nhl())

[('1994-05-31', 'Stanley Cup Finals - Game 1', 'New York Rangers', 2),
 ('1994-05-31', 'Stanley Cup Finals - Game 1', 'Vancouver Canucks', 3),
 ('1994-06-02', 'Stanley Cup Finals - Game 2', 'New York Rangers', 3),
 ('1994-06-02', 'Stanley Cup Finals - Game 2', 'Vancouver Canucks', 1),
 ('1994-06-04', 'Stanley Cup Finals - Game 3', 'New York Rangers', 5),
 ('1994-06-04', 'Stanley Cup Finals - Game 3', 'Vancouver Canucks', 1),
 ('1994-06-07', 'Stanley Cup Finals - Game 4', 'New York Rangers', 4),
 ('1994-06-07', 'Stanley Cup Finals - Game 4', 'Vancouver Canucks', 2),
 ('1994-06-09', 'Stanley Cup Finals - Game 5', 'New York Rangers', 3),
 ('1994-06-09', 'Stanley Cup Finals - Game 5', 'Vancouver Canucks', 6),
 ('1994-06-11', 'Stanley Cup Finals - Game 6', 'New York Rangers', 1),
 ('1994-06-11', 'Stanley Cup Finals - Game 6', 'Vancouver Canucks', 4),
 ('1994-06-14', 'Stanley Cup Finals - Game 7', 'New York Rangers', 3),
 ('1994-06-14', 'Stanley Cup Finals - Game 7', 'Vancouver Canucks', 2)]

In [88]:
execute_sql(select_date_info_and_total_points_for_highest_scoring_nba_game())

[('1994-06-10', 'NBA Finals - Game 2', 91)]